In [ ]:
# WEEDMAPS DATA CLEANING: BUSINESS-LEVEL DATA
# - create indicators for whether we have medical / recreational license info
# - create indicators for descriptors (e.g. in-store purchases only, order online, medical and recreational etc)
# - create variable for state that dispensary is located in
# - split license variable into medical and recreational licenses (INCOMPLETE)


# Imports
import pandas as pd
import os
import re
from pyzipcode import ZipCodeDatabase


In [ ]:
zcdb = ZipCodeDatabase()

In [ ]:
# Import business-level data
df = pd.read_csv("../stores_data.csv")

Create indicators for medical / recreational license


In [ ]:
cats = ["Medical", "Adult-Use"]
for cat in cats:
    df[cat + "_ID?"] = df["Licenses"].str.contains(cat, case = False).astype(float)

Create indicator variables based on descriptors

In [ ]:
df2 = df.copy()
df2["Descriptors"] = df2.apply(lambda row : "|".join([x.strip() for x in row["Descriptors"].split("|")]),
                             axis = 1)

In [ ]:
df_desc2 = df2.Descriptors.str.get_dummies()

In [ ]:
df3 = df2.join(df_desc2)

Create state variable

In [ ]:
states =["Alabama", "Alaska", "Arizona", "Arkansas", "California", 
             "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", 
             "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
             "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", 
             "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", 
             "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", 
             "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", 
             "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
             "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", 
             "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

# function to check which state a dispensary is in
def get_state(col_val):
    lst = []
    for state in states:
        st = state.lower() + " "
        if st in col_val.lower():
            lst.append(state)
    if len(lst) == 1:
        return [lst[0]]
    elif len(lst) > 1:
        # need to change this to check by zipcode
        return [x for x in lst]

In [ ]:
# applying function to dataframe
df4 = df3.copy()
df4["State"] = df4.apply(lambda row : get_state(row["Address"]),
                             axis = 1)

In [ ]:
# businesses with 2 potential state names in their address (currently 167)
dftest = df4[~df4.State.isna()]
df_amb = dftest[dftest.State.map(len) > 1]
len(df_amb)

In [ ]:
# businesses which are likely not based in any of the 50 US states (currently 1480)
df_no_state = df4[df4["State"].isna()]
len(df_no_state)

In [ ]:
# testing
test = df.Descriptors[0]
test2 = df2.Descriptors[0]
test2
test

In [ ]:
test = df.Address[0]
test

In [ ]:
# CREATE INDICATORS BASED ON DESCRIPTORS
# from sklearn.feature_extraction.text import CountVectorizer

# cv = CountVectorizer()

# df_desc = pd.DataFrame(cv.fit_transform(df.Descriptors.fillna('').str.replace(r"\|", " ")).A,
#                        columns=cv.get_feature_names_out(),
#                        index = df.index)

In [ ]:
# df4[df4["State"].isna() & ~df4["Address"].str.contains("ON ")]
# df4[df4["Address"].str.contains("ON ")]